<a href="https://colab.research.google.com/github/olakoja/DA-MUS/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%92%D1%8B%D0%B3%D1%80%D1%83%D0%B7%D0%BA%D0%B0_%D1%82%D1%80%D0%B5%D0%BA%D0%BE%D0%B2_%D0%B8%D0%B7_OSM_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Выгрузка и подготовка данных о gps-треках из OSM

Эта часть посвящена выгрузке данных о gps-треков с OSM.
Необходимо подключать свой ГуглДиск, так как выгружает большое количество данных и при сохранение в среде выполнения они могут нанести вред выполнению кода или быть повреждены сам. По Твери с пространства 0,5х0,5 градусов выгрузилось 787 файлов в СПб значительно больше. Размер одного выгружаемого файла — 5000 объектов.

In [ ]:
!pip install gpxpy -qq
import requests
import os
from google.colab import drive
import math
from geopy.distance import geodesic
import geopandas as gpd

In [ ]:
# Монтируем Google Диск
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1. Загрузка GeoDataFrame из файла
input_file = "/content/All_Predictions.gpkg"  # В ковычках файл с территорией, на которую нужны данные
gdf = gpd.read_file(input_file)

if gdf.crs != 4326:
    gdf = gdf.to_crs(4326)

# 2. Получаем крайние точки из геодатафрейма
min_lon, min_lat, max_lon, max_lat = gdf.total_bounds

# Функция для вычисления площади в квадратных километрах
def calculate_bbox_area(lat1, lon1, lat2, lon2):
    # Вычисляем длину сторон в километрах
    length = geodesic((lat1, lon1), (lat1, lon2)).kilometers
    width = geodesic((lat1, lon1), (lat2, lon1)).kilometers
    area = length * width
    return area

# Вычисляем размеры в градусах
width_deg = max_lon - min_lon  # Ширина по долготе
height_deg = max_lat - min_lat  # Высота по широте

# Вычисляем площадь участка
area = calculate_bbox_area(min_lat, min_lon, max_lat, max_lon)
print(f"Площадь участка: {area:.5f} км²")
print(f"Или размер участка: {width_deg:.5f}° (долгота) × {height_deg:.5f}° (широта)")

# URL для запроса с координатами
url = f"https://api.openstreetmap.org/api/0.6/trackpoints?bbox={min_lon},{min_lat},{max_lon},{max_lat}&page={{}}"

# Путь к папке на Google Диске
folder_path = '/content/drive/My Drive/gps_osm1'

page_number = 1647

# Проверяем ограничение в 0.5 градуса
MAX_SIZE_DEG = 0.5

if width_deg > MAX_SIZE_DEG or height_deg > MAX_SIZE_DEG:
    print(f"Ошибка: Размер участка превышает {MAX_SIZE_DEG}° по одной из осей")
    print("Загрузка отменена")
else:
    print("Размер участка допустимый, начинаем загрузку...")

    # Очистка папки, если она существует и не пуста
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
            except Exception as e:
                print(f"Ошибка при удалении файла {file_path}: {e}")
        print(f"Папка {folder_path} очищена")
    else:
        os.makedirs(folder_path)
        print(f"Папка {folder_path} создана")

    while True:
        current_url = url.format(page_number)
        response = requests.get(current_url)

        if response.status_code == 200:
            file_name = f"{page_number}.gpx"
            file_path = os.path.join(folder_path, file_name)

            with open(file_path, "wb") as file:
                file.write(response.content)

            file_size = os.path.getsize(file_path)

            if file_size < 137:
                print(f"Файл {file_name} имеет размер {file_size} байт (меньше 137 байт). Удаляем и останавливаем цикл.")
                os.remove(file_path)
                break

            print(f"Данные сохранены в файле: {file_path} (размер: {file_size} байт)")
            page_number += 1
        else:
            print("Произошла ошибка при загрузке данных.")
            break

    print("Загрузка завершена")

Площадь участка: 79.23086 км²
Или размер участка: 0.19666° (долгота) × 0.06449° (широта)
Размер участка допустимый, начинаем загрузку...
Папка /content/drive/My Drive/gps_osm1 создана
Данные сохранены в файле: /content/drive/My Drive/gps_osm1/1647.gpx (размер: 315180 байт)
Данные сохранены в файле: /content/drive/My Drive/gps_osm1/1648.gpx (размер: 315180 байт)
Данные сохранены в файле: /content/drive/My Drive/gps_osm1/1649.gpx (размер: 315180 байт)
Данные сохранены в файле: /content/drive/My Drive/gps_osm1/1650.gpx (размер: 315180 байт)
Данные сохранены в файле: /content/drive/My Drive/gps_osm1/1651.gpx (размер: 315180 байт)
Данные сохранены в файле: /content/drive/My Drive/gps_osm1/1652.gpx (размер: 315180 байт)
Данные сохранены в файле: /content/drive/My Drive/gps_osm1/1653.gpx (размер: 315180 байт)
Данные сохранены в файле: /content/drive/My Drive/gps_osm1/1654.gpx (размер: 315180 байт)
Данные сохранены в файле: /content/drive/My Drive/gps_osm1/1655.gpx (размер: 315180 байт)
Данные

# Обработка gpx-файлов для получения DataFrame

Установка необходимых бибилиотек.
* библиотеки для чтения gpx;
* gdal (при некорректной работе gdal никакая последующая работа будет невозможна)

И проверка установки gdal

In [ ]:
from google.colab import drive

# Монтируем Google Диск
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gpxpy -qq

import gpxpy
import gpxpy.gpx
!pip install gdal -qq

In [ ]:
import importlib

def check_module(module_name):
    try:
        importlib.import_module(module_name)
        print(f"The {module_name} module is installed.")
    except ImportError:
        print(f"The {module_name} module is not installed.")

check_module('osgeo')  # Попытка импорта gdal через osgeo

The osgeo module is installed.


Этот код позволит получить данные в формате DataFrame. Это будет  таблица без геометрии. В ней данные о времени и координатах каждой точки в gps-треке.

In [ ]:
# Есть риск, что вы не будете ждать окончания загрузки, сидя перед монитором. Поэтому мы снова импортируем необходимые библиотеки

import os
import gpxpy
import pandas as pd

In [ ]:
import gpxpy
import pandas as pd
import os
from datetime import datetime

# 1. Укажите путь к вашей папке с GPX-файлами
folder_path = '/content/drive/My Drive/gps_osm1'  # Если код не будет оставлен, путь к папке можно сделать комментарием (поставить #)

# 2. Собираем все GPX-файлы из папки
gpx_files = [f for f in os.listdir(folder_path) if f.endswith('.gpx')]

# 3. Создаем список для хранения данных
data = []

# 4. Обрабатываем каждый GPX-файл
for gpx_file in gpx_files:
    file_path = os.path.join(folder_path, gpx_file)

    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            gpx = gpxpy.parse(f)

            # Извлекаем все точки треков
            for track in gpx.tracks:
                for segment in track.segments:
                    for point in segment.points:
                        # Добавляем точку в список данных
                        data.append({
                            'latitude': point.latitude,
                            'longitude': point.longitude,
                            'elevation': point.elevation if point.elevation is not None else 0,
                            'time': point.time if point.time else pd.NaT,
                            'filename': gpx_file  # Добавляем имя файла для отслеживания
                        })
        except Exception as e:
            print(f"Ошибка при обработке файла {gpx_file}: {e}")

# 5. Создаем датафрейм из собранных данных
df = pd.DataFrame(data)

# 6. Проверяем результат
print(f"Всего обработано точек: {len(df)}")
print(df.head())

Всего обработано точек: 131859
    latitude  longitude  elevation time  filename
0  59.902581  30.317912          0  NaT  1647.gpx
1  59.902581  30.317915          0  NaT  1647.gpx
2  59.902581  30.317918          0  NaT  1647.gpx
3  59.902581  30.317918          0  NaT  1647.gpx
4  59.902581  30.317921          0  NaT  1647.gpx


Сохраняем DataFrame в файл формата .csv

In [ ]:
# 1. Определяем путь к папке и имя файла отдельно
folder_path = '/content/drive/My Drive/gps_osm1'  # Только путь к папке
filename = 'gpx_points.csv'  # Имя файла

# 2. Создаем папку, если её нет (без проверки имени файла)
os.makedirs(folder_path, exist_ok=True)

# 3. Объединяем путь при сохранении
output_path = os.path.join(folder_path, filename)
df.to_csv(output_path, index=False)

print(f'Файл сохранен: {output_path}')

Файл сохранен: /content/drive/My Drive/gps_osm1/gpx_points.csv


# Далее идёт обработка DataFrame, который мы сохраняли в файл.
Если среда выполнения была заброшена, можно снова загрузить всё необходимое: библиотеки, файл.

In [ ]:
import pandas as pd
import geopandas as gpd

Загрузка и просмотр данных файла

In [ ]:
# Полный путь к файлу
 # output_path = os.path.join(folder_path, filename)

# filename = "/content/drive/MyDrive/filtered_tracks/tracks_point.csv"
df = pd.read_csv(output_path, delimiter=",")

df.head(5)

,latitude,longitude,elevation,time,filename
0,59.902581,30.317912,0,NaN,1647.gpx
1,59.902581,30.317915,0,NaN,1647.gpx
2,59.902581,30.317918,0,NaN,1647.gpx
3,59.902581,30.317918,0,NaN,1647.gpx
4,59.902581,30.317921,0,NaN,1647.gpx


Просмотр сведений о файле.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131859 entries, 0 to 131858
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   latitude   131859 non-null  float64
 1   longitude  131859 non-null  float64
 2   elevation  131859 non-null  int64  
 3   time       0 non-null       float64
 4   filename   131859 non-null  object 
dtypes: float64(3), int64(1), object(1)
memory usage: 5.0+ MB


Сохраняем необходимые столбцы

In [ ]:
columns = [
    "time",
    "latitude",
    "longitude"
]

df = df.loc[:, columns]
df.head(5)

,time,latitude,longitude
0,NaN,59.902581,30.317912
1,NaN,59.902581,30.317915
2,NaN,59.902581,30.317918
3,NaN,59.902581,30.317918
4,NaN,59.902581,30.317921


Добавляем id, как отдельный столбец

In [ ]:
df['id'] = df.index

Проверяем результат

In [ ]:
df.set_index("id", inplace=True)
df.head(5)

,time,latitude,longitude
id,,,
0,NaN,59.902581,30.317912
1,NaN,59.902581,30.317915
2,NaN,59.902581,30.317918
3,NaN,59.902581,30.317918
4,NaN,59.902581,30.317921


Делим столбец времени на дату и время отдельно




In [ ]:
df_1 = (
    df["time"].astype(str)  # Конвертируем в строку
    .str.split(" ", expand=True)  # Теперь split() сработает
)
df_1['id'] = df_1.index

df_1

,0,id
id,,
0,nan,0
1,nan,1
2,nan,2
3,nan,3
4,nan,4
...,...,...
131854,nan,131854
131855,nan,131855
131856,nan,131856


Даём столбцам новые названия

In [ ]:
# Проверяем, есть ли столбец 1 в df_1, если нет - создаем пустой
if 1 not in df_1.columns:
    df_1[1] = "0"  # или pd.NA для пропущенных значений

# Переименовываем столбцы
df_0 = df_1.rename(columns={0: "day", 1: "uhr"})

df_0.head(5)

,day,id,uhr
id,,,
0,nan,0,
1,nan,1,
2,nan,2,
3,nan,3,
4,nan,4,


Объединяем со старыми данными

In [ ]:
df_2 = df.join(df_0)

df_2.head(5)

,time,latitude,longitude,day,id,uhr
id,,,,,,
0,NaN,59.902581,30.317912,nan,0,
1,NaN,59.902581,30.317915,nan,1,
2,NaN,59.902581,30.317918,nan,2,
3,NaN,59.902581,30.317918,nan,3,
4,NaN,59.902581,30.317921,nan,4,


Смотрим, что получилось

In [ ]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 131859 entries, 0 to 131858
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   time       0 non-null       float64
 1   latitude   131859 non-null  float64
 2   longitude  131859 non-null  float64
 3   day        131859 non-null  object 
 4   id         131859 non-null  int64  
 5   uhr        131859 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 7.0+ MB


Оставляем только нужное

In [ ]:
columns = [
    "day",
    "uhr",
    "latitude",
    "longitude"
]

df_3 = df_2.loc[:, columns]
df_3.head()

,day,uhr,latitude,longitude
id,,,,
0,nan,,59.902581,30.317912
1,nan,,59.902581,30.317915
2,nan,,59.902581,30.317918
3,nan,,59.902581,30.317918
4,nan,,59.902581,30.317921


Импортируем в файл

In [ ]:
# 1. Определяем путь к папке и имя файла отдельно
folder_path = '/content/drive/MyDrive/filtered_tracks'  # Только путь к папке
filename = 'time1.tracks_point.csv'  # Имя файла

# 2. Создаем папку, если её нет (без проверки имени файла)
os.makedirs(folder_path, exist_ok=True)

# 3. Объединяем путь при сохранении
output_path = os.path.join(folder_path, filename)
df_3.to_csv(output_path, index=False)

print(f'Файл сохранен: {output_path}')

Файл сохранен: /content/drive/MyDrive/filtered_tracks/time1.tracks_point.csv


# Получаем из нового DataFrame GeoDataFrame.
Если удобнее, это можно сделать инструментами QGis (Слой-->Добавить слой-->Добавить слой из текста с разделителями...)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
# Если снова забросили код, то вот строка для импорта
# df_3 = "/content/drive/MyDrive/filtered_tracks/time.tracks_point.csv"

# 1. Создаем геометрию из координат
geometry = [Point(lon, lat) for lon, lat in zip(df_3['longitude'], df_3['latitude'])]

# 2. Создаем GeoDataFrame
gdf = gpd.GeoDataFrame(
    df_3.drop(['longitude', 'latitude'], axis=1),  # Исходные данные без координатных колонок
    geometry=geometry,
    crs="EPSG:4326"  # Указываем WGS84 (широта/долгота в градусах)
)

# Проверяем результат
print(gdf.head())
print(f"CRS: {gdf.crs}")

    day uhr                   geometry
id                                    
0   nan      POINT (30.31791 59.90258)
1   nan      POINT (30.31792 59.90258)
2   nan      POINT (30.31792 59.90258)
3   nan      POINT (30.31792 59.90258)
4   nan      POINT (30.31792 59.90258)
CRS: EPSG:4326


In [ ]:
gdf.to_file('gdf.tracks.gpkg', driver="GPKG")